In [ ]:
include("../load.jl")

In [ ]:
using Plots
using LaTeXStrings
using JuMP

In [ ]:
# Creating the GlobalModel
m = JuMP.Model(with_optimizer(CPLEX_SILENT))
# @variable(m, -20 <= x[1:2] <= 20)
# set_lower_bound(x[2], 0)
# set_upper_bound(x[2], 10)
@variable(m, 1 <= x[1:2] <= 5)
@variable(m, y)
gm = GlobalModel(model = m)
# add_nonlinear_constraint(gm, :(x -> x[1]^2 + x[2]^2), dependent_var = y)
# add_nonlinear_constraint(gm, :(x -> log(x[1]^2 + x[2]^2)), dependent_var = y)
# add_nonlinear_constraint(gm, :(x -> exp(- 1/6*(x[1]^2) - 1/10*(x[2]^2))), dependent_var = y)
# add_nonlinear_constraint(gm, :(x -> x[1] + x[2]), dependent_var = y)
# add_nonlinear_constraint(gm, :(x -> x[1]^4*x[2]^2 + x[1]^2*x[2]^4 - 3*x[1]^2*x[2]^2 + 1), dependent_var = y)
add_nonlinear_constraint(gm, :(x -> max(0, x[1] - 2 - 0.1*x[1]*x[2]^2, x[2] - 2)), dependent_var = y)
bbr = gm.bbls[end]
set_param(bbr, :n_samples, 500)
uniform_sample_and_eval!(gm, lh_iterations=20)
classify_curvature(bbr)

In [ ]:
# Curvature plot
bbr = gm.bbls[1]
plt3d = Plots.plot()
for val in [-1, 0, 1]
    idxs = findall(i -> i == val, bbr.curvatures)
    Plots.scatter!(bbr.X[idxs,"x[2]"], bbr.X[idxs,"x[1]"], bbr.Y[idxs], markersize = 2, label = val, camera = (10,50))
end
xlabel!(L"$x_2$"); ylabel!(L"$x_1$"); title!(L"$y$")
display(plt3d)

In [ ]:
# We can train trees over the curvatures
lnr = base_classifier()
kwargs = Dict()
nl = OCTHaGOn.learn_from_data!(bbr.X, bbr.curvatures .> 0, lnr; fit_classifier_kwargs(; kwargs...)...)

In [ ]:
# Plotting convexity
preds = IAI.predict(nl, bbr.X);
one_preds = findall(x -> x == 1, preds)
zero_preds = findall(x -> x == 0, preds)
Plots.scatter(bbr.X[one_preds,"x[2]"], bbr.X[one_preds,"x[1]"], bbr.Y[one_preds], 
            markersize = 2, seriescolor=:blue, camera = (10,50))
Plots.scatter!(bbr.X[zero_preds,"x[2]"], bbr.X[zero_preds,"x[1]"], bbr.Y[zero_preds], 
            markersize = 2, seriescolor=:red, camera = (10,50))

In [ ]:
# How about univariate example? 
m = JuMP.Model()
@variable(m, -5 <= x <= 10)
@variable(m, y)
gm = GlobalModel(model = m)
# add_nonlinear_constraint(gm, :(x -> max.(-6*x .-6,0.5x,0.2x.^5+0.5x)), dependent_var = y)
# add_nonlinear_constraint(gm, :(x -> x^1.2*sin(x)), dependent_var = y)
add_nonlinear_constraint(gm, :(x -> min((x-1)^2, (x+2)^2)), dependent_var = y)
uniform_sample_and_eval!(gm)
classify_curvature(gm.bbls[1])

In [ ]:
bbr = gm.bbls[1]
plt2d = Plots.plot()
for val in [-1, 0, 1]
    idxs = findall(i -> i == val, bbr.curvatures)
    Plots.scatter!(bbr.X[idxs,"x"], bbr.Y[idxs], markersize = 4, label = val)
end
display(plt2d)

In [ ]:
learn_constraint!(bbr)

In [ ]:
bbr.learners[end]